In [1]:
import uproot
import pandas as pd

In [17]:
f_cppFramework = uproot.open("/eos/user/z/zhenxuan/DoubleHiggs/MC_Root/GluGluToRadionToHHTo2G4Q_M300.root")

In [18]:
f_cppFramework.keys()

['Events;4',
 'Events;3',
 'TotalEvents;1',
 'totalCutFlow_FH_OnlyFourjetsCategory;1',
 'totalCutFlow_FH;1',
 'Radion_Cutflow;1',
 'Ak4JetsCut;1',
 'totalCutFlow_FH_GENMatch;1',
 'totalCutFlow_SL;1',
 'eventNumber;1']

In [19]:
cut_table_cppFramework = f_cppFramework["totalCutFlow_FH_OnlyFourjetsCategory"].values()

In [27]:
# get all events eff with combine all the log file together
import os 
import numpy as np
cmd_events = 'grep "load_events" /eos/user/z/zhenxuan/hhwwgg/GluGluToRadionToHHTo2G4Q_M300_2017/*/*.log'
cmd_HHWWtagger_events = 'grep "my_HHWW_Preselection : event set" /eos/user/z/zhenxuan/hhwwgg/GluGluToRadionToHHTo2G4Q_M300_2017/*/*.log'
cmd_photon = 'grep "Photon Selection" /eos/user/z/zhenxuan/hhwwgg/GluGluToRadionToHHTo2G4Q_M300_2017/*/*.log'
cmd_lepton = 'grep "Lepton Selection" /eos/user/z/zhenxuan/hhwwgg/GluGluToRadionToHHTo2G4Q_M300_2017/*/*.log'
cmd_jet = 'grep "OnlyFourJet_category" /eos/user/z/zhenxuan/hhwwgg/GluGluToRadionToHHTo2G4Q_M300_2017/*/*.log'
events = os.popen(cmd_events).readlines()
HHWWtagger_events = os.popen(cmd_HHWWtagger_events).readlines()
photon = os.popen(cmd_photon).readlines()
lepton = os.popen(cmd_lepton).readlines()
jet = os.popen(cmd_jet).readlines()
events_number = []
HHWWtagger_events_number = []
Photon_eff = []
Lepton_eff = []
Jet_eff = []
Jet_eff = []
for i in range(len(events)):
    events_number.append(int(events[i].split("Loaded")[-1].split("events")[0]))
    HHWWtagger_events_number.append(float(HHWWtagger_events[i].split('(')[0].split()[-1]))
    Photon_eff.append(float(HHWWtagger_events[i].split('(')[0].split()[-1]) * float(photon[i].split(':')[-2].split(' ')[1])/int(events[i].split("Loaded")[-1].split("events")[0]) )
    Lepton_eff.append(float(HHWWtagger_events[i].split('(')[0].split()[-1]) * float(lepton[i].split(':')[-2].split(' ')[1])/int(events[i].split("Loaded")[-1].split("events")[0]) )
    Jet_eff.append(float(HHWWtagger_events[i].split('(')[0].split()[-1]) * float(jet[i].split(':')[-2].split(' ')[1])/int(events[i].split("Loaded")[-1].split("events")[0]) )
    # Jet_eff.append(float(jet[i].split(':')[-2].split(' ')[1]))
Photon_eff = float(np.mean(Photon_eff))
Lepton_eff = float(np.mean(Lepton_eff))
Jet_eff = float(np.mean(Jet_eff))
all_HHWWtagger_events = float(np.sum(HHWWtagger_events_number))
all_events = float(np.sum(events_number))
print("all events: %i" %(all_events) )
print("Photon numbers:%i" %(all_events*Photon_eff), "Photon Selection eff: %f" %(Photon_eff) )
print("Lepton numbers:%i" %(all_events*Lepton_eff), "Lepton Selection eff: %f" %(Lepton_eff) )
print("Jet numbers:%i" %(all_events*Jet_eff), "Jet Selection eff: %f" %(Jet_eff) )


all events: 288160
Photon numbers:108592 Photon Selection eff: 0.376850
Lepton numbers:108489 Lepton Selection eff: 0.376492
Jet numbers:43027 Jet Selection eff: 0.149319


In [56]:
df_cut_table = pd.DataFrame()
df_cut_table["cpp framework N"] = [cut_table_cppFramework[0],cut_table_cppFramework[1],cut_table_cppFramework[2],cut_table_cppFramework[3]]
df_cut_table["cpp framework eff(%)"] = [str(round(cut_table_cppFramework[0]/cut_table_cppFramework[0] * 100,2)) + '%' ,str(round(cut_table_cppFramework[1]/cut_table_cppFramework[0] * 100,2)) + '%', str(round(cut_table_cppFramework[2]/cut_table_cppFramework[0] * 100,2)) + '%' , str(round(cut_table_cppFramework[3]/cut_table_cppFramework[0] * 100,2)) + '%' ]
df_cut_table["higgsdna framework N"] = [round(all_events,0),round(all_events*Photon_eff,0), round(all_events*Lepton_eff,0) ,round(all_events*Jet_eff,0)]
df_cut_table["higgsdna framework eff(%)"] = [str(round(all_events / all_events * 100,2)) + '%',str(round(Photon_eff* 100,2)) + '%', str(round(Lepton_eff* 100,2)) + '%',str(round(Jet_eff* 100,2)) + '%' ]

In [52]:
df_cut_table = pd.DataFrame()
df_cut_table["all Events"] = [cut_table_cppFramework[0],str(round(cut_table_cppFramework[0]/cut_table_cppFramework[0] * 100,2)) + '%' , all_events,str(all_events/all_events * 100)+'%']
df_cut_table["Photon selection"] = [cut_table_cppFramework[1],str(round(cut_table_cppFramework[1]/cut_table_cppFramework[0] * 100,2)) + '%', round(all_events*Photon_eff,0), str(round(Photon_eff* 100,2)) + '%']
df_cut_table["Lepton selection"] = [cut_table_cppFramework[2],str(round(cut_table_cppFramework[2]/cut_table_cppFramework[0] * 100,2)) + '%',round(all_events*Lepton_eff,0),str(round(Lepton_eff* 100,2)) + '%']
df_cut_table["Only ak4 jets selection"] = [cut_table_cppFramework[3],str(round(cut_table_cppFramework[3]/cut_table_cppFramework[0] * 100,2)) + '%' ,round(all_events*Jet_eff,0),str(round(Jet_eff* 100,2)) + '%' ]


In [57]:
df_cut_table.index = ['all Events', 'Photon selection','Lepton Selection', 'Only ak4 jets slection']

In [59]:
df_cut_table.to_csv("only 4 jets eff check")

In [ ]:
f_higgsdna = uproot.open("/eos/user/z/zhenxuan/DoubleHiggs/MC_Root/GluGluToRadionToHHTo2G4Q_M300.root")